# AIF360 Inprocessing bias Mitigation with sklearn-compatible interface

In this notebook I present an example of in-processing bias mitigation for classification using the Adult dataset. The method tested is the reductions approach via ExponentiatedGradientReduction, GridSearchReduction

In [1]:
from aif360.sklearn.datasets import fetch_adult
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
import numpy as np
from aif360.sklearn.inprocessing import ExponentiatedGradientReduction, GridSearchReduction
from aif360.sklearn.metrics import equal_opportunity_difference
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [2]:
np.random.seed(0) #for reproducibility

## 1. Load data

In [3]:
X, y, sample_weight = fetch_adult()
X.head()

/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


age  workclass     education  education-num  \
race      sex                                                  
Non-white Male  25.0    Private          11th            7.0   
White     Male  38.0    Private       HS-grad            9.0   
          Male  28.0  Local-gov    Assoc-acdm           12.0   
Non-white Male  44.0    Private  Some-college           10.0   
White     Male  34.0    Private          10th            6.0   

                    marital-status         occupation   relationship   race  \
race      sex                                                                 
Non-white Male       Never-married  Machine-op-inspct      Own-child  Black   
White     Male  Married-civ-spouse    Farming-fishing        Husband  White   
          Male  Married-civ-spouse    Protective-serv        Husband  White   
Non-white Male  Married-civ-spouse  Machine-op-inspct        Husband  Black   
White     Male       Never-married      Other-service  Not-in-family  White   

                 sex  capital-gain  capital-loss  hours-per-week  \
race      sex                                                      
Non-white Male  Male           0.0           0.0            40.0   
White     Male  Male           0.0           0.0            50.0   
          Male  Male           0.0           0.0            40.0   
Non-white Male  Male        7688.0           0.0            40.0   
White     Male  Male           0.0           0.0            30.0   

               native-country  
race      sex                  
Non-white Male  United-States  
White     Male  United-States  
          Male  United-States  
Non-white Male  United-States  
White     Male  United-States

In [4]:
print(f"Dataset dimensions: {X.shape}")

Dataset dimensions: (45222, 13)


## 2. Preprocessing

Group multiple categories for race into white and non-white.

In [5]:
X.race = X.race.cat.set_categories(['Non-white', 'White'], ordered=True).fillna('Non-white')

Set index and label as integers.

In [6]:
X.index = pd.MultiIndex.from_arrays(X.index.codes, names=X.index.names)
y.index = pd.MultiIndex.from_arrays(y.index.codes, names=y.index.names)
y = pd.Series(y.factorize(sort=True)[0], index=y.index)

Split into train and test

In [7]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, train_size=0.7)

Transform categories into one hot encoding

In [8]:
ohe = make_column_transformer(
        (OneHotEncoder(sparse=False), X_train.dtypes == 'category'),
        remainder='passthrough', verbose_feature_names_out=False)
X_train  = pd.DataFrame(ohe.fit_transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
X_test = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)

X_train.head()

/Users/claudia/repositories/datascience-explained/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


workclass_Federal-gov  workclass_Local-gov  workclass_Private  \
race sex                                                                  
0    0                      0.0                  0.0                1.0   
1    1                      0.0                  0.0                1.0   
     1                      0.0                  0.0                1.0   
     1                      0.0                  0.0                0.0   
     1                      0.0                  0.0                0.0   

          workclass_Self-emp-inc  workclass_Self-emp-not-inc  \
race sex                                                       
0    0                       0.0                         0.0   
1    1                       0.0                         0.0   
     1                       0.0                         0.0   
     1                       1.0                         0.0   
     1                       0.0                         1.0   

          workclass_State-gov  workclass_Without-pay  education_10th  \
race sex                                                               
0    0                    0.0                    0.0             0.0   
1    1                    0.0                    0.0             0.0   
     1                    0.0                    0.0             0.0   
     1                    0.0                    0.0             0.0   
     1                    0.0                    0.0             0.0   

          education_11th  education_12th  ...  native-country_Thailand  \
race sex                                  ...                            
0    0               0.0             0.0  ...                      0.0   
1    1               0.0             0.0  ...                      0.0   
     1               0.0             0.0  ...                      0.0   
     1               0.0             0.0  ...                      0.0   
     1               0.0             0.0  ...                      0.0   

          native-country_Trinadad&Tobago  native-country_United-States  \
race sex                                                                 
0    0                               0.0                           1.0   
1    1                               0.0                           0.0   
     1                               0.0                           1.0   
     1                               0.0                           1.0   
     1                               0.0                           1.0   

          native-country_Vietnam  native-country_Yugoslavia   age  \
race sex                                                            
0    0                       0.0                        0.0  47.0   
1    1                       0.0                        0.0  24.0   
     1                       0.0                        0.0  19.0   
     1                       0.0                        0.0  43.0   
     1                       0.0                        0.0  47.0   

          education-num  capital-gain  capital-loss  hours-per-week  
race sex                                                             
0    0             12.0           0.0           0.0            40.0  
1    1             10.0           0.0           0.0            10.0  
     1              9.0           0.0           0.0            40.0  
     1             10.0           0.0           0.0            50.0  
     1             10.0           0.0           0.0            30.0  

[5 rows x 100 columns]

## 3. Baseline model

Train baseline logistic regression

In [9]:
y_pred = LogisticRegression(solver='liblinear').fit(X_train, y_train).predict(X_test)
lr_acc = accuracy_score(y_test, y_pred)
lr_acc

0.84145352694037

Compute equal opportunity difference.

In [10]:
lr_eod_sex = equal_opportunity_difference(y_test, y_pred, prot_attr='sex')
lr_eod_sex

-0.12786633836183348

We see that it is positive, for sex, indicating that the true positive rate is higher for females.

In [11]:
lr_eod_race = equal_opportunity_difference(y_test, y_pred, prot_attr='race')
lr_eod_race

-0.052776993150201235

We see that it is negative, for race, indicating that the true positive rate is higher for white.

In [12]:
estimator = LogisticRegression(solver='liblinear')
prot_attr_cols = [colname for colname in X_train if "sex" in colname or "race" in colname]

## 4. Bias mitigation: ExponentiatedGradientReduction inprocessing

Exponentiated gradient reduction is an in-processing technique that reduces fair classification to a sequence of cost-sensitive classification problems, returning a randomized classifier with the lowest empirical error subject to fair classification constraints

Train a model with in-processing bias mittigation, enforcing parity in TPR.

In [13]:
exp_grad_red = ExponentiatedGradientReduction(prot_attr=prot_attr_cols,
                                              estimator=estimator,
                                              constraints="TruePositiveRateParity",
                                              drop_prot_attr=False)
exp_grad_red.fit(X_train, y_train)
egr_acc = exp_grad_red.score(X_test, y_test)
print(egr_acc)

# Check for that accuracy is comparable
assert abs(lr_acc-egr_acc)<=0.03

0.8341564089334415


We see that there is no drop in accuracy. Now we analyze the equal opportunity difference

In [14]:
egr_eod_sex = equal_opportunity_difference(y_test, exp_grad_red.predict(X_test), prot_attr='sex')
print(egr_eod_sex)

# Check for improvement in average odds error for sex
assert abs(egr_eod_sex)<abs(lr_eod_sex)

-0.015318005334953533


In [15]:
egr_eod_race = equal_opportunity_difference(y_test, exp_grad_red.predict(X_test), prot_attr='race')
print(egr_eod_race)

# Check for improvement in average odds error for race
assert abs(egr_eod_race)<abs(lr_eod_race)

0.006934759197796736


## 4. Bias mitigation: GridSearchReduction inprocessing

Grid search is an in-processing technique that can be used for fair classification or fair regression. For classification it reduces fair classification to a sequence of cost-sensitive classification problems, returning the deterministic classifier with the lowest empirical error subject to fair classification constraints among the candidates searched. For regression it uses the same priniciple to return a deterministic regressor with the lowest empirical error subject to the constraint of bounded group loss

In [16]:
exp_grad_red = GridSearchReduction(prot_attr=prot_attr_cols,
                                              estimator=estimator,
                                              constraints="TruePositiveRateParity",
                                              drop_prot_attr=False)
exp_grad_red.fit(X_train, y_train)
egr_acc = exp_grad_red.score(X_test, y_test)
print(egr_acc)

0.7623645610672957


For this method, we get a lower accuracy